In [1]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [4]:
# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [5]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07), metrics=['accuracy'])


In [6]:
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)


Epoch 1/50


448/448 [==============================] - 306s 678ms/step - loss: 1.7940 - accuracy: 0.2670 - val_loss: 1.6826 - val_accuracy: 0.3530
Epoch 2/50
448/448 [==============================] - 278s 620ms/step - loss: 1.6121 - accuracy: 0.3737 - val_loss: 1.5306 - val_accuracy: 0.4234
Epoch 3/50
448/448 [==============================] - 363s 810ms/step - loss: 1.5085 - accuracy: 0.4173 - val_loss: 1.4421 - val_accuracy: 0.4541
Epoch 4/50
448/448 [==============================] - 135s 301ms/step - loss: 1.4332 - accuracy: 0.4547 - val_loss: 1.3818 - val_accuracy: 0.4771
Epoch 5/50
448/448 [==============================] - 133s 296ms/step - loss: 1.3794 - accuracy: 0.4739 - val_loss: 1.3380 - val_accuracy: 0.4869
Epoch 6/50
448/448 [==============================] - 124s 276ms/step - loss: 1.3273 - accuracy: 0.4932 - val_loss: 1.2941 - val_accuracy: 0.5039
Epoch 7/50
448/448 [==============================] - 128s 285ms/step - loss: 1.2866 - accuracy: 0.5132 - val_loss: 1.2606

In [7]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)


In [8]:
# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')